In [50]:
import requests, json
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
import calendar
from datetime import datetime
import time
import sys
import random

#converts numpy types to python types, otherwise json conversion produces an error. call json.dumps(***, cls=MyEncoder)
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

In [51]:
#input values. Read config.txt

#set up which stations to parse here
listtoparse=[]
#listtoparse.append('DEBY006') #DEBY006 Augsburg Königsplatz
#listtoparse.append('DEBY007') #DEBY007 Augsburg Bourges Platz
#listtoparse.append('DEBY008') #DEBY008 Augsburg Haunstetten
#listtoparse.append('DEBY081') #DEBY081 Garmisch Kreuzeckbahnstr
#listtoparse.append('DEBY082') #DEBY082 Garmisch Wankgipfel
#listtoparse.append('DEBY083') #DEBY083 Garmisch Zugspitzgipfel - no information about sensors
#listtoparse.append('DEBY099') #DEBY099 Augsburg LfU
#listtoparse.append('DEBY110') #DEBY110 Augsburg Karlstraße
#listtoparse.append('DEBY123') #DEBY123 Garmisch am Herrgottschrofen
#listtoparse.append('DEBY150') #DEBY150 Augsburg DBS - no information about sensors
#listtoparse.append('DEBY196') #DEBY196 Garmisch Wasserwerk
#add more to the list to parse


try:
    configfile=open('config.txt')
    config=json.loads(configfile.read())

    url = config['url']
    listtoparse.append(config['thingcode'])
    feature = config['feature']
#    intervalllength = config['intervalllength']
except:
    sys.exit("Config File not properly set!")


upload=True #set to True/False to enable/disable upload of metadata to FROST


#url = 'http://smartaqnet-dev.teco.edu:8080/FROST-Server/v1.0'
#feature='PM10'



baseurl = 'https://www.umweltbundesamt.de/js/uaq/data/stations' #get data where from
scope='1SMW' #umweltbundesamt website code: 1 hour means
scopesec= 60*60 #scope in seconds, needed for interval to tag the next observation

print("Upload is set to " + str(upload))
print("Extracting Observations for ObservedProperty " + str(feature))

Upload is set to True
Extracting Observations for ObservedProperty PM10


In [52]:
## to check which stations contain which observedproperty
#fulllist=[]
#for ind in range(len(list(filemeta["component_code"]))):
#    if list(filemeta["component_code"])[ind] == 5.0:
#        fulllist.append([list(filemeta["station_code"])[ind],list(filemeta["parameter"])[ind]])
#
#fulllist

In [53]:
#input parameters

listofreplacements=[
    ('PM10','5.0'),
    ('PM2,5','6001.0'),
    ('PM1','6002.0'),
]

#fetches the code for the above defined 'feature'
for eachelement in listofreplacements:
    if eachelement[0] == feature:
        code = eachelement[1]


file = pd.read_excel('metadata/Bericht_EU_Meta_Stationen.xlsx')
filemeta=pd.read_excel('metadata/Bericht_EU_Meta_Stationsparameter.xlsx')
df_stationparameters = filemeta.set_index("station_code")

#federal state, network code, website
ccodesetreadable=[('Hessen', 'DE009A','hlnug.de'),
 ('Saarland', 'DE001A','saarland.de'),
 ('Berlin', 'DE008A','berlin.de'),
 ('Bayern', 'DE007A','lfu.bayern.de'),
 ('Rheinland-Pfalz', 'DE011A','luft.rlp.de'),
 ('Sachsen', 'DE016A','umwelt.sachsen.de'),
 ('Umweltbundesamt', 'DE006A','umweltbundesamt.de'),
 ('Baden-Wuerttemberg', 'DE005A','lubw.baden-wuerttemberg.de'),
 ('Nordrhein-Westfalen', 'DE004A','lanuv.nrw.de'),
 ('Brandenburg', 'DE014A','lfu.brandenburg.de'),
 ('Bremen', 'DE013A','bauumwelt.bremen.de'),
 ('Mecklenburg-Vorpommern', 'DE018A','lung.mv-regierung.de'),
 ('Hamburg', 'DE012A','luft.hamburg.de'),
 ('Thueringen', 'DE017A','tlug-jena.de'),
 ('Schleswig-Holstein', 'DE002A','schleswig-holstein.de'),
 ('Sachsen-Anhalt', 'DE015A','luesa.sachsen-anhalt.de'),
 ('Niedersachsen', 'DE010A','umwelt.niedersachsen.de')]


#crosscheck
for element in listtoparse:
    print('Check: Going to uploading data for station ' + str(element) + ' at ' + str(file.set_index("station_code")["station_name"][element]))


Check: Going to uploading data for station DEBY007 at Augsburg/Bourges-Platz


In [54]:
#returns the url for the network code
def repnetcodebyurl(netcode):
    for item in ccodesetreadable:
        if item[1]==netcode:
            return(item[2])

#returns the name of the state for the network code
def repnetcodebystate(netcode):
    for item in ccodesetreadable:
        if item[1]==netcode:
            return(item[0])

#------------------------------------------------------------------------------------
#functions for time conversion
def readtime(sec):
    m, s = divmod(sec, 60)
    h, m = divmod(m, 60)
    return(str(int(h)) + " hours " + str(int(m)) + " minutes " + str(int(s)) + " seconds")

def tounixtime(datetime_input):
    return(calendar.timegm(datetime_input.utctimetuple()))

def todatetimeformat(utctime):
    year=int(utctime[0])*1000 + int(utctime[1])*100 + int(utctime[2])*10 + int(utctime[3])
    month=int(utctime[5])*10 + int(utctime[6])
    day=int(utctime[8])*10 + int(utctime[9])
    hr=int(utctime[11])*10 + int(utctime[12])
    minute=int(utctime[14])*10 + int(utctime[15])
    second=int(utctime[17])*10 + int(utctime[18])
    millisecond=int(utctime[20])*100 + int(utctime[21])*10 + int(utctime[22])   
    return(datetime(year,month,day,hr,minute,second,millisecond))

def toutcformat(datetime_input):
    tstr=str(datetime_input)
    year=tstr[0]+tstr[1]+tstr[2]+tstr[3]
    month=tstr[5]+tstr[6]
    day=tstr[8]+tstr[9]
    
    try:
        if type(int(tstr[11]+tstr[12]))==int:
            hour=str(tstr[11]+tstr[12])
    except:
        hour='00'             #no hours given       
    
    try:
        if type(int(tstr[14]+tstr[15]))==int:
            minute=str(tstr[14]+tstr[15])
    except:
        minute='00'             #no minutes given

    try:
        if type(int(tstr[17]+tstr[18]))==int:
            second=str(tstr[17]+tstr[18])
    except:
        second='00'             #no seconds given
        
    try:
        if type(int(tstr[20]+tstr[21]+tstr[22]))==int:
            millisecond=str(tstr[20]+tstr[21]+tstr[22])
    except:
        millisecond='000'       #no milliseconds given


        
    utctime=year + '-' + month + '-' + day + 'T' + hour + ':' + minute + ':' + second + '.' + millisecond + 'Z'
    return utctime

def addminutesutc(utctime,mins):
    return(toutcformat(datetime.utcfromtimestamp(tounixtime(todatetimeformat(utctime))+(mins*60))))

currentyear=str(datetime.utcnow())[0:4]

In [ ]:
#metadata
totalstarttime = time.time() #to check how long the upload took

#observed property
generatepropertyid= "saqn:o:" + str(feature).lower().replace(" ", "_")


obsproperty = {
    "name": str(feature),
    "description": "",
    "definition": "",
    "@iot.id": str(generatepropertyid).lower().replace(" ", "_")
    }
if upload==True:
    requests.post(url + '/ObservedProperties', json.dumps(obsproperty))
else:
    pass

#list over all stations in the given list

for thingcode in listtoparse:
    
    thingnr=list(file["station_code"]).index(thingcode) #the number of the row in the excel file
    
    generatedescr="" #generates the description for the thing
    if  df_stationparameters["type_of_parameter"].index.contains(thingcode): #checks whether the station actually exists
        if thingcode in df_stationparameters["type_of_parameter"][thingcode]: #if the stations measures only one type of parameter, the index is not returned somehow and the loop produces an error, therefore check if the index is returned and if not handle case separately
            for element in list(set(df_stationparameters["type_of_parameter"][thingcode])):
                generatedescr+= " -" + element + "-"
        else: #if the station only measures one type of parameter (loop produces an error in that case, thus handled separately)
            generatedescr+= " -" + df_stationparameters["type_of_parameter"][thingcode] + "-"

    #------------------------------------------------------------------------------------------------
    #building things
    
    #Location ID
    generatelocid = "geo:" + str(float(file["station_longitude_d"][thingnr])) + "," + str(float(file["station_latitude_d"][thingnr])) + "," + str(float(file["station_altitude"][thingnr]))
    
    #Thing ID
    generatethingid="saqn:t" #generates the id for the thing by adding each of the following features to define it uniquely
    generatethingid+=":" + str(repnetcodebyurl(file["network_code"][thingnr])) #adds the url through the network code
    generatethingid+=":" + str(file["station_name"][thingnr])
    generatethingid+=":" + "comm" + str(file["station_start_date"][thingnr])[0:6]
    generatethingid+=":" + str(file["station_code"][thingnr])
#    generatethingid+=":" + str(currentyear) #adds the current year to make the identifier unique in case the id gets changed one day
#    for label in ["station_name","station_start_date","station_code"]: #add more info if desired
#        generatethingid+=":" + str(file[label][thingnr])

    #generates a dictionary of all raw properties of the thing
    rawproperties = {}
    for eachproperty in list(file):
        if str(file[eachproperty][thingnr])=='nan':
            rawproperties[eachproperty]='nan'
        else:
            rawproperties[eachproperty] = file[eachproperty][thingnr]

    #generate the thing JSON
    thingdata = {"name": "Measuring Station " + str(thingcode),
        "description": "A station measuring" + str(generatedescr),
        "properties": rawproperties,
        "@iot.id": str(generatethingid).lower().replace(" ", "_").replace("/", "_"),
         "Locations": [{
            "name": "Location of " + "measuring Station " + str(thingcode),
            "description": "located at " + str(file["station_name"][thingnr]),
            "encodingType": "application/vnd.geo+json",
            "@iot.id": str(generatelocid),
            "location": {
                  "type": "Point",
                  "coordinates": [float(file["station_latitude_d"][thingnr]), float(file["station_longitude_d"][thingnr]), float(file["station_altitude"][thingnr])]
            }
             
          }]
    }
    if upload==True:
        requests.post(url + '/Things', json.dumps(thingdata, cls=MyEncoder))
    else:
        pass


    
    #loop over all sensors and check which contains the requested observedproperty
    try:
        if float(code) not in list(df_stationparameters["component_code"][thingcode]):
            print("Station " + str(thingcode) + " does not measure " + str(feature))
        for sensnr in range(len(list(df_stationparameters["component_code"][thingcode]))):
            if list(df_stationparameters["component_code"][thingcode])[sensnr] == float(code):
                thissensor=list(df_stationparameters["parameter"][thingcode])[sensnr] #the parameter to parse, e.g. "Particulate Matter - PM10, first measurement"
                mestech=list(df_stationparameters["measurement_technique_principle"][thingcode])[sensnr]
                #warning: if only one sensor exists, this will blow up because one item is not returned as dataframe but as string

                #------------------------------------------------------------------------------------------------
                #building the sensors
                #generates a dictionary of all raw properties of the thing to dump into metadata property
                rawmetadata = {}
                rawmetadata["station_code"]=thingcode
                for eachdata in list(df_stationparameters): #option 1: all metadata
                #for eachdata in ["type_of_parameter","parameter","component_code","measurement_technique_principle"]: #option 2: pick
                    if str(list(df_stationparameters[eachdata][thingcode])[sensnr]) == 'nan':
                        rawmetadata[eachdata]='nan'
                    else:
                        rawmetadata[eachdata] = list(df_stationparameters[eachdata][thingcode])[sensnr]


                #Sensor ID - example: saqn:s:lfu.bayern.de:particulate_matter_-_pm10_first_measurement:nephelometry_and_beta_attenuation:2019:deby007

                generatesensorid="saqn:s" #generates the id for the sensor by adding each of the following features to define it uniquely
                generatesensorid+=":" + str(repnetcodebyurl(file["network_code"][thingnr])) #adds the url through the network code
                generatesensorid+=":" + "unknown_type_" + str(list(df_stationparameters["measurement_technique_principle"][thingcode])[sensnr]) + "_sensor"


    #                for label in ["parameter","measurement_technique_principle"]: #add more info if desired
    #                    generatesensorid+=":" + str(list(df_stationparameters[label][thingcode])[sensnr])
    #                generatesensorid+=":" + str(currentyear) #adds the current year to make the identifier unique in case the id gets changed one day
    #                generatesensorid+=":" + str(thingcode) #code of the corresponding station

                #generate sensor JSON
                sensor = {"name": "A " + str(feature) + " sensor",
                        "description": "A sensor measuring " + str(feature) + " using " + str(mestech),
                        "encodingType": "application/json",
                        "metadata": "",
                        "@iot.id": str(generatesensorid).lower().replace(" ", "_")
                        }
                if upload==True:
                    requests.post(url + '/Sensors', json.dumps(sensor, cls=MyEncoder))
                else:
                    pass

                #------------------------------------------------------------------------------------------------
                #building the datastreams

                generatestreamid = "saqn:d"
                generatestreamid+=":" + str(repnetcodebyurl(file["network_code"][thingnr]))
                generatestreamid+=":" + "unknown_type_" + str(list(df_stationparameters["measurement_technique_principle"][thingcode])[sensnr]) + "_sensor"
                generatestreamid+=":" + "comm" + str(list(df_stationparameters["measurement_start_date"][thingcode])[sensnr])[0:6]
                generatestreamid+=":" + str(thingcode)
                generatestreamid+=":" + str(feature)

                print("Building Datastream " + str(generatestreamid).lower().replace(" ", "_"))

                datastream = {"name": str(thissensor) + " Datastream of station " + str(thingcode),
                            "description": "A Datastream measuring " + str(thissensor) + " using " + str(mestech),
                            "observationType": "",
                            "unitOfMeasurement": {
                                "name": "microgram per cubic meter",
                                "symbol": "ug/m^3",
                                "definition": "none"
                                },
                            "properties": rawmetadata,
                            "@iot.id": str(generatestreamid).lower().replace(" ", "_"),
                            "Thing":{"@iot.id":str(generatethingid).lower().replace(" ", "_").replace("/", "_")},
                            "Sensor":{"@iot.id":str(generatesensorid).lower().replace(" ", "_")},
                            "ObservedProperty":{"@iot.id":str(generatepropertyid).lower().replace(" ", "_")}
                            }

                if upload==True:
                    requests.post(url + '/Datastreams', json.dumps(datastream))
                else:
                    pass

                #END OF METADATA - - - BEGIN OF DATA UPLOAD
                #------------------------------------------------------------

                def graburl(start,end): #start and end in datetime
                    return(baseurl + '/measuring?pollutant[]=' + feature + '&scope[]=' + scope + '&station[]=' + thingcode + '&group[]=pollutant&range[]=' + str(tounixtime(start)) + ',' + str(tounixtime(end)))

                #begin time
                try:
                    begintime=todatetimeformat(config['starttime'])
                except:

                    #get the start month of the respective datastream

                    def getstarttime():
                        for i in range(1970,2018+1): #ranges to 2018, the +1 is because how range counts
                            try:
                                theurl=graburl(datetime(i,1,1,0,0,0),datetime(i,12,31,23,59,0))
                                datafromurl=json.loads(requests.get(theurl).content)["data"][0]
                                if datafromurl[0][0]!='bananas': #anything but an error
                                    for j in range(1,12+1):
                                        try:
                                            theurl2=graburl(datetime(i,j,1,0,0,0),datetime(i,j,calendar.monthrange(i,j)[1],23,59,0))
                                            datafromurl2=json.loads(requests.get(theurl2).content)["data"][0]
                                            if datafromurl2[0][0]!='bananas':
                                                for k in range(1,calendar.monthrange(i,j)[1]+1):
                                                    try:
                                                        theurl3=graburl(datetime(i,j,k,0,0,0),datetime(i,j,k,23,59,0))
                                                        datafromurl3=json.loads(requests.get(theurl3).content)["data"][0]
                                                        if datafromurl3[0][0]!='bananas':
                                                            return(datetime(i,j,k,0,0,0))
                                                            break
                                                    except:
                                                        pass
                                        except:
                                            pass
                            except:
                                pass

                    begintime=getstarttime()
                    print("Parsing start time not properly set. Taking earliest date measurements appear: " + str(begintime))

                #end time
                try:
                    endtime=todatetimeformat(config['endtime'])
                except:
                    print("Parsing end time not properly set. Taking " + str(datetime.utcnow()))
                    endtime  =datetime.utcnow() 


                #------------------------------------------------------------
                print('Measurements of ' + str(thingcode) + ' ' +  str(thissensor) + ' are being parsed from ' + str(begintime))

                begintimeunix=tounixtime(begintime)
                endtimeunix=tounixtime(endtime)

                getdatafrom=baseurl + '/measuring?pollutant[]=' + feature + '&scope[]=' + scope + '&station[]=' + thingcode + '&group[]=pollutant&range[]=' + str(begintimeunix + (scopesec/2)) + ',' + str(endtimeunix  + (scopesec/2))
                datavalue=json.loads(requests.get(getdatafrom).content)["data"][0]


                #convert list into a dataframe
                datalist=[]
                labels=['interval_start_time','interval_end_time',str(feature)]

                for i in range(len(datavalue)):
                        datalist.append([toutcformat(datetime.utcfromtimestamp(begintimeunix + (scopesec*i))),toutcformat(datetime.utcfromtimestamp(begintimeunix + ((scopesec*i) + (scopesec-60)) )),datavalue[i][0]])

                dataframe = pd.DataFrame.from_records(datalist, columns=labels)
                dataframemeta = pd.DataFrame.from_records([[generatestreamid.lower().replace(" ", "_")]], columns=['datastreamID'])

                #save dataframe to excel sheet
                filename= str(thingcode) + "_" + str(thissensor) + "_" + str(toutcformat(begintime)[0:10]) + "_" + str(toutcformat(endtime)[0:10]) + ".xlsx"
                writer = ExcelWriter('data/' +  str(filename))
                dataframe.to_excel(writer,'Sheet1',index=False)
                dataframemeta.to_excel(writer,'id',index=False)
                writer.save()

                #crosschecking: checks a number of random points between the dataframe and the url

                #the UBA database gives the result for the hour that has PASSED, e.g. the value at 5 o'clock is the mean between 4 and 5 o'clock
                #the code writes a value for the interval between e.g. 3:00 and 3:59. for this interval, the UBA value at 3:30 is taken. 
                #therefore the check also needs to add 30m to the url to check with the corresponding start time

                numberoftests=10
                testistrue=False


                for i in range(0,10):
                    r = random.randint(1,len(list(dataframe["interval_start_time"])))
                    getdatapoint=graburl(todatetimeformat(addminutesutc(dataframe["interval_start_time"][r],30)),todatetimeformat(addminutesutc(dataframe["interval_end_time"][r],30)))
                    checkdatavalue=json.loads(requests.get(getdatapoint).content)["data"][0][0][0]
                    if dataframe[feature][r] == checkdatavalue:
                        testistrue=True
                    else:
                        print(str(dataframe[feature][r]) + "!=" + str(checkdatavalue))
                        testistrue=False

                print("Crosscheck for " + str(thingcode) + " " + str(thissensor) + " with " + str(numberoftests) + " random datapoints gave result " + str(testistrue))
    except:
        print("Could not retrieve any data for station " + str(thingcode))
    
    
endtime=time.time()
timeelapsed=endtime-totalstarttime
    
print("Time elapsed: " + str(readtime(timeelapsed)))

if config['runparseaftermodelling']==True:
    exec(open('UBA_parse_excel.py').read())
    Print['Feeding Measurements into ' + str(url)]

Building Datastream saqn:d:lfu.bayern.de:unknown_type_nephelometry_and_beta_attenuation_sensor:comm201412:deby007:pm10
Parsing start time not properly set. Taking earliest date measurements appear: 2016-12-31 00:00:00
Measurements of DEBY007 Particulate matter - PM10, first measurement are being parsed from 2016-12-31 00:00:00
Crosscheck for DEBY007 Particulate matter - PM10, first measurement with 10 random datapoints gave result True
Time elapsed: 0 hours 0 minutes 43 seconds
Loading file data\DEBY007_Particulate matter - PM10, first measurement_2016-12-31_2018-12-31.xlsx...
Uploading Observations for Datastream iot.id: saqn:d:lfu.bayern.de:unknown_type_nephelometry_and_beta_attenuation_sensor:comm201412:deby007:pm10
